# Animating Atmospheric Rivers 
#### Lander Greenway, Cole Pietsch
***
>Version 1.1.2

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import ftplib
import os
import imageio.v3 as iio
import datetime
from glob import glob

#### *Define parameters for animation*

This progarm works with NetCDF files. Available variables are tp, tcwv, tcc.

In [2]:
# Define start and end dates
start = '2023-06-22'
end = '2023-06-25'

#define var as the mapped variable, string name inside array(?)
var = 'tcc'

In [3]:
# load data
data = xr.open_dataset('/Users/colepietsch/Documents/EV333_AtmosphericDynamics/Rivers/atmorivers/ERA5_SouthAmerica_2023_hourly_tcc.nc')

***

In [4]:
#output directory
output_dir = str(var + '_' + start + '_' + end)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
else:
    print("Directory '{}' already exists. Skipping creation.".format(output_dir))

In [5]:
#subset data and extract max for scaling color bar
data = data.sel(time = slice(start, end))
max_value = data[var].max().values.item()

if not (-0.5 <= max_value <= 0.5) :
    round(max_value)

tick_value = max_value / 40 if round(max_value / 40) == 0 else round(max_value / 40)

    

#extracting variable, applying color bar contraint by %, and applying unit conversions if needed (also convert units of tick and max values)
if var == 'tp' :
    data = data[var] * 1000
    title = 'Total Precipitation'
    lev = np.arange(0, 1000 * max_value * 0.4, tick_value * 1000) 
    cmap = cmocean.cm.rain  
    cbar_lab = "Total Precip. in mm/hr"
    xcoord_style = "longitude"
    ycoord_style = "latitude"

elif var == 'tcwv' :
    data = data[var]
    title = 'Total Column Water Vapor'
    lev = np.arange(0, max_value * 0.6, tick_value)
    cmap = cmocean.cm.deep
    cbar_lab = "TCWV (kg/m^2)"
    xcoord_style = "longitude"
    ycoord_style = "latitude"

elif var == 'tcc' :
    data = data[var]
    title = 'Total Cloud Cover'
    lev = np.arange(0, 1.05, 0.05)
    cmap = cmocean.cm.ice_r
    cbar_lab = "TCC (%)"
    xcoord_style = "longitude"
    ycoord_style = "latitude"

else:
    print('ERROR: You Done Screwed Up. Variable not recognized')

#defining number of frames given by subset data
num = len(data['time'])

In [6]:
#choosing projection
proj = ccrs.PlateCarree(central_longitude=-77)

#create loop
for i in range(0, num) :

    # define figure and axes, figure size, and resolution
    fig = plt.figure(figsize=(8, 4.5), dpi=300)
    ax = plt.axes(projection = proj)

    # filled contour map of metric in question
    data[i].plot.contourf(
        x = xcoord_style, 
        y = ycoord_style,
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=lev,
        extend='max',
        colors=cmap,
        add_colorbar=True,
        cbar_kwargs = {"label":cbar_lab})

    # add coastlines
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.coastlines(resolution='110m')  #Currently can be one of “110m”, “50m”, and “10m”.

    # add grid lines
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True,
                      linewidth=1,
                      color='gray',
                      alpha=0.5,
                      linestyle='--')
    gl.right_labels = None
    gl.top_labels = None
    

    # Extract year, month, day, and hour from time series, splits are used on format of 'YYYY-MM-DDT00:00:00.000000000'
    times = data['time'][i].values.astype('datetime64[s]')
    yr = str(np.datetime64(times, 'Y').astype(str))
    mon = str(np.datetime64(times, 'M').astype(str).split('-')[1])
    day = str(np.datetime64(times, 'D').astype(str).split('-')[2])
    hr = str(np.datetime64(times, 'h').astype(str).split('T')[1].split(':')[0])
   
    # Construct the filename using the extracted information
    name = output_dir + f'/{var}_{yr}-{mon}-{day}-{hr}.png'

    # add title
    ax.set_title(title + ' on ' + mon + '/' + day + ', ' + yr + ' at ' + hr + ':00')
  
    # Save the figure
    fig.savefig(name, facecolor='white', transparent=False, bbox_inches='tight')


    
    plt.close(fig)


In [7]:
#getting frames 
files = sorted(glob(output_dir + '/' + var + '*.png'))

#creating empty array
images = [ ]

#setting loop to append files to images
for filename in files :
    images.append(iio.imread(filename))

#GIF output directory
gifDir = "OutputGifs"
if not os.path.exists(gifDir):
    os.mkdir(gifDir)
else:
    print("Directory '{}' already exists. Skipping creation.".format(gifDir))

#writing gif, placing it in the GIF output directory
iio.imwrite("OutputGifs/" + var + '_' + start + '_' + end +'.gif', images, duration = 150, loop = 0)

Directory 'OutputGifs' already exists. Skipping creation.


In [8]:
#clearing the directory frames are stored in 
if os.path.exists(output_dir):
    #iterate for all items in directory
    for item in os.listdir(output_dir):
        item_path = os.path.join(output_dir, item)  
        
        # Check if it's a file and delete it
        if os.path.isfile(item_path):
            os.remove(item_path)
            
        # Check if it's a directory and delete it recursively
        elif os.path.isdir(item_path):
            os.rmdir(item_path)
            
    # After deleting all files and directories within the directory, remove the directory itself
    os.rmdir(output_dir)
    print(f"Directory '{output_dir}' deleted successfully.")
    
else:
    print("Directory does not exist.")

Directory 'tcc_2023-06-22_2023-06-25' deleted successfully.
